# Splice site prediction

Most eukaryotic genes undergo intron splicing events. These happen at the level of mRNA, after transcription and before translation. Pre-mRNA consists of exons, which are the proteincoding regions, and introns, which are the intervening sections in between. As a base rule, the spliceosome cuts out introns, retaining the consecutive exons to form mature mRNA. This mature mRNA is then translated to protein.

In this practical you will build a classification model for gene splice site prediction from DNA sequences. The vast majority of splice sites are characterized by the presence of specific dimers on the intronic side of the splice site: "GT" for donor and "AG" for acceptor sites. Yet, only about 0.1-1% of all "GT" and "AG" occurrences in the genome represent true splice sites. 

We will focus on acceptor site prediction. You are given the following data set:

In [0]:
import pandas as pd

data = pd.read_csv("https://raw.githubusercontent.com/sdgroeve/Machine_Learning_course_UGent_D012554_data/master/practicum/Classification/acceptor_sites_dataset_train.csv")

In [0]:
data.head(5)

There are only two columns. The column "sequence" contains a local DNA context sequence that surrounds a candidate acceptor site (nucleotides at positions 11 and 12 in the sequence are always "A" and "G" respectively), so these positions are candidate gene acceptor sites. The column "label" indicates the class: 1 for "is acceptor site" and -1 for "is not acceptor site". The goal is to predict the target from the local context sequence of the candidate acceptor site. 

*How many sequences does the dataset contain for each class?*

In [0]:
###Start code here

###End code here

Next, useful features need to be computed from the DNA sequences, a process known as **feature engineering**. 

The Pandas `.apply()` method allows us the process the values in a DataFrame column to create a new column. We will apply this function to compute feature vectors from the `sequence` column in the `data` DataFrame.  

The "AG" in the middle of each context sequence is the same for both classes, i.e. it does not provide any discriminative information. So, We shouldn't compute features from the middle AG dinculeotide in the local context sequence.

*Use the Pandas DataFrame `.apply()` method to remove the middle "AG" dinucleotides in the DNA sequences:*

In [0]:
print(data.head())

###Start code here

###End code here

data["sequence"] = data["sequence"].apply(remove_AG)
print(data.head())

A trivial feature vector representation would be to replace each amino acid with a number, making the Machine Learning aware about each amino acid at each position in the DNA context sequence. The following method maps a DNA sequence `x`to a Python list with a number for each amino acid:

In [0]:
def DNA_int_encoding(x):
    encoding = []
    for nuc in x:
        if nuc == 'A':
            encoding.append(0)
        elif nuc == 'C':
            encoding.append(1)
        elif nuc == 'G':
            encoding.append(2)
        elif nuc == 'T':
            encoding.append(3)
        else:
            print("Found non-nucleotide in %s"%x)
    return encoding

*Apply this function on the `sequence` column in the `data` DataFrame to create a Series with feature vectors (write the resulting feature vectors to a variable called `data_features_int_encoding`:*

In [0]:
###Start code here
data_features_int_encoding = 
###End code here

data_features_int_encoding.head()

Next we put these feature vectors back in a Pandas DataFrame as follows:

In [0]:
data_features_int_encoding = pd.DataFrame(data_features_int_encoding.tolist())

data_features_int_encoding.head()

Evaluate the generalization performance of a logisitc regression model with hyperparameters $C=0.1$ on the data set `data_features_int_encoding` using 10-fold cross-validation. 

*Apply the `cross_val_score()` function to compute an accuracy score for each fold in the CV:*

In [0]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
import numpy as np

###Start code here
model = 
scores = 
###Start code here

print(np.mean(scores))

Next we load an independent test set:

In [0]:
data_test = pd.read_csv("https://raw.githubusercontent.com/sdgroeve/Machine_Learning_course_UGent_D012554_data/master/practicum/Classification/acceptor_sites_dataset_test.csv")

*Compute the feature vectors for the test set:*

In [0]:
###Start code here (try to use just one line of code)
data_test_features_int_encoding = 
###Start code here

*Fit a logistic regression model on the train set.*

In [0]:
###Start code here

###End code here

*Make model predictions for the test set.*

In [0]:
###Start code here
predictions = 
###End code here

Scikit-learn offers many metrics to evaluate model predictions. These functions are contained in the `metrics` module of `sklearn`. 

*Can you find how to compute the accuracy of these predictions (use the `metrics`module)?*

In [0]:
from sklearn import metrics

###Start code here
score_acc = 
###End code here

print(score_acc)

An accuracy above 90% seems like a good score. But is it? Let's consider a model that predicts class "-1" for all test points.

In [0]:
predictions_zero = [-1]*len(data_test.label)

*What is the accuracy of these predictions?*

In [0]:
###Start code here
score_acc = 
###End code here

print(score_acc)

So this should be a good score as well, even though we did not learn anything.

For classification tasks where the classes are highly imbalanced accuracy is not a good metric to evaluate the generalization performance. In fact, if there are 0.1% "AG" dinucleotides in a genome that are true acceptor sites then a model that predicts class "-1" for each "AG" would have an accuracy of 99.9%.

We have seen how a ROC curve plots the true positives rate against the false positives rate. Both these metrics focus on the positive class, in our case the true acceptor sites. These metrics are much more suitable to evalute the performance of models on tasks with highly imbalanced classes. To transform a ROC curve into one metric we can use the area under the curve (AUC). 

*What is the AUC score of the predictions computed by the linear regression model we fitted?*

In [0]:
###Start code here
score_auc = 
###End code here

print(score_acc)

You should see a negative value. This is because to compute the AUC we need the predictions to be scores (a continuous value) rather than class labels (discrete values). 

For logistic regression these scores are the class probabilities predicted by the model. We can obtain them using the `predict_proba()` function of the `LogisticRegression` module as follows:

In [0]:
predictions = model.predict_proba(data_test_features_int_encoding)

print(predictions)

The first and second column contain the predicted probabilities for class '-1' and '1' respectively. To compute the AUC we need to use the positive class probabilities. 

*What is the AUC now?*

In [0]:
###Start code here
score_auc = 
###End code here

print(score_auc)

Is this good generalization performance?

Transforming categorical features into ordered integers is maybe not a good idea as the nucleotides don't have any ordering. It is better to transform a categorical feature into one binary feature for each category (known as *one-hot* encoding). 

We can do this with the following function that again computes feature vectors:

In [0]:
def DNA_onehot_encoding(x):
    encoding = []
    for nuc in x:
        if nuc == 'A':
            encoding.extend([1,0,0,0])
        elif nuc == 'C':
            encoding.extend([0,1,0,0])
        elif nuc == 'G':
            encoding.extend([0,0,1,0])
        elif nuc == 'T':
            encoding.extend([0,0,0,1])
        else:
            encoding.extend([0,0,0,0])
    return encoding

*Create a Pandas DataFrame called `data_features_onehot_encoding` that contains the one-hot encoded features.*

In [0]:
###Start code here
data_features_onehot_encoding = 
###End code here

data_features_onehot_encoding.head()

Now let's add interpretable feature names:

In [0]:
columns = []
for i in range(-10,0,1):
    for nuc in ['A','C','G','T']:
        columns.append("%i_%s"%(i,nuc))
for i in range(1,11,1):
    for nuc in ['A','C','G','T']:
        columns.append("%i_%s"%(i,nuc))       
data_features_onehot_encoding.columns = columns
data_features_onehot_encoding.head()

Evaluate the generalization performance of a logisitc regression model with hyperparameters $C=0.1$ on the data set `data_features_onehot_encoding` using 10-fold cross-validation. 
The `cross_val_score()` has a function parameter called `scoring` that allows you to set different scoring metrics.

*Use the `cross_val_score()` function to compute the mean AUC of the CV-scores.* 


In [0]:
model = LogisticRegression(C=0.1)

###Start code here
score_acc = 
###End code here

print(score_auc)

*What is the AUC on `data_test`?*

In [0]:
###Start code here

###Start code here

print(score_acc)

Is this close to what your CV is telling you?

We have used hyperparameter $C=0.1$ for the logistic regression model. 

*Is there a better value for this regularization parameter (use `GridSearchCV`)?*

In [0]:
from sklearn.model_selection import GridSearchCV

search_space = [0.001,0.01,0.1,1,10,100]

###Start code here

params = 
grid_search = 

###End code here

print(grid_search.best_estimator_)

*What is the 10-CV AUC performance with this value for $C$?*

In [0]:
###Start code here

###Start code here

print(score_auc)

*What is the AUC performance on the test set for this value of $C$?*

In [0]:
###Start code here

###End code here

print(score_auc)

Is this closer to the AUC you computed using 10-CV?

Let's see how the model is making predictions (what patterns it extracted). In scikit-learn a fitted model has its modelparameters stored in `.coef_[0]`:

In [0]:
print(model.coef_[0])

For logistic regression this is one modelparameter for each feature (plus the interecept, which is not in `.coef_[0]`). 

Recall that for logistic regression a prediction is made by multiplying each fitted modelparameter with the corresponding feature, summing them and then squeezing this sum between 0 and 1 through the logistic function. Since all features have values 0 or 1 the modelparameter values indicate the importance of a feature during prediction.

For plotting we will put the feature names and modelparameter values in a new DataFrame:

In [0]:
F_importances = []
for feature_name, modelparameter in zip(data_features_onehot_encoding.columns,model.coef_[0]):
    F_importances.append([feature_name,modelparameter])
F_importances = pd.DataFrame(F_importances,columns=["feature_name","importance"])
F_importances.head()    

*Use the Seaborn `.barplot()` method to plot this DataFrame:*

In [0]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(18,8))

###Start code here
chart = sns.
###End code here

chart.set_xticklabels(chart.get_xticklabels(), rotation=90, horizontalalignment='right')
plt.show()

*Create a plot that looks like this:*

![plot](https://raw.githubusercontent.com/sdgroeve/Machine_Learning_course_UGent_D012554_data/master/practicum/Classification/AG_plot.png)

In [0]:
###Start code here

###End code here